# HuggingFace로 기계번역 구현하기

이번에는 HuggingFace로 기계번역 모델을 학습해 볼 것입니다.
먼저 필요한 library들을 설치하고 import합시다.

In [1]:
%pip install transformers datasets evaluate accelerate scikit-learn sacrebleu sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 41.1 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.4/22.4 MB 67.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 65.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import random
import evaluate
import numpy as np

from datasets import load_dataset
from transformers import AutoTokenizer

## Dataset 준비

기계번역을 위한 dataset을 준비하겠습니다.
Dataset은 `Helsinki-NLP/opus-100`을 활용합니다.

In [3]:
books = load_dataset("Helsinki-NLP/opus-100", "en-ko")
books["train"][1]

README.md:   0%|          | 0.00/65.4k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/143k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/70.1M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/144k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

{'translation': {'en': "I ain't fishing' 'em out.", 'ko': '그거 꺼내려다가는'}}

보시다시피 각 data는 영어 문장을 `en`에, 한국어 문장을 `ko`에 저장하고 있습니다.

이번에는 tokenizer를 불러와 data를 미리 tokenize 하겠습니다.

In [4]:
source_lang = "en"
target_lang = "ko"
prefix = "translate English to Korean: "
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-tc-big-en-ko")

def preprocess_function(data):
    inputs = [prefix + text[source_lang] for text in data["translation"]]
    targets = [text[target_lang] for text in data["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

tokenized_books = books.map(preprocess_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/790k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/815k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/959k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

기계번역과 같이 입력과 출력을 모두 tokenize하는 경우에는 출력 text를 `tokenizer`의 `text_target`인자로 넘겨주면 됩니다. 이처럼 기계 번역 dataset을 불러오는 것은 imdb dataset과 별반 다르지 않습니다.

마지막으로 `data_collator`를 다음과 같이 구현합니다.

In [5]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model='google-t5/t5-small')

## Model 구현

이번에는 기계번역을 위한 sequence-to-sequence 모델을 구현하겠습니다.

In [6]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained('google-t5/t5-small')

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Sequence-to-sequence 모델은 위와 같이 `AutoModelForSeq2SeqLM`을 활용하면 됩니다.
Pre-trained 모델로 `google-t5/t5-small`이라는 것을 사용하고 있습니다.
이 코드 하나로 우리는 기계 번역과 같은 문제를 풀 수 있는 encoder-decoder 구조의 모델을 구현할 수 있습니다.

## 학습

마지막으로 학습 코드를 구현하겠습니다.
학습은 감정 분석과 똑같이 `training_args`를 정의하고 `Trainer`로 이전에 구현한 것들을 모두 수합하면 됩니다.

In [7]:
training_args = Seq2SeqTrainingArguments(
    output_dir="hf_mt",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    weight_decay=0.01,  # 첫 주차 때 배운 weight decay를 조절하는 hyper-parameter입니다.
    save_total_limit=1,
    num_train_epochs=1,
    predict_with_generate=True  # 실제로 평가를 진행할 때는 내부적으로 정의된 beam search 등을 활용하여 text를 생성합니다.
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_books["train"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

/var/folders/sm/f3jq7h1d5ms4gybg0zfs83fm0000gn/T/ipykernel_12236/2895406638.py:11: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


마지막으로 학습하면 다음과 같습니다.

In [8]:
trainer.train()
trainer.save_model()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss


KeyboardInterrupt: 

학습 결과를 보면 다음과 같습니다.

In [ ]:
from transformers import pipeline

text = "translate English to Korean: Hello! My name is Seunghyuk Cho."
translator = pipeline("translation", model='./hf_mt', max_new_tokens=128, device='cuda', tokenizer=tokenizer)
translator(text)

[{'translation_text': '-의 그 그 그 그 그 그 그 그 그 그 그 그...................................................'}]

보시다시피 성능이 그렇게 좋지 않습니다. 아마 더 많은 학습 시간을 요구하는 듯 합니다.

하지만 중요한 것은 data부터 모델, 학습 코드 구현까지 매우 쉽게 했다는 것입니다.